In [1]:
%run intro.py

In [1]:
import fastai
from fastai.vision import *
import torch
from data import *
from utils import *
from fastai.callbacks import *
from fastai.utils.mem import *
from mish.mxresnet import mxresnet34, mxresnet50

# Seed everything
seed_value = 42
import random 
random.seed(seed_value) # Python
import numpy as np
np.random.seed(seed_value) # cpu vars
import torch
torch.manual_seed(seed_value) # cpu  vars

if torch.cuda.is_available(): 
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value) # gpu vars
    torch.backends.cudnn.deterministic = True  #needed
    torch.backends.cudnn.benchmark = False

Mish activation loaded...
Mish activation loaded...


In [2]:
from types import SimpleNamespace

In [3]:
args = SimpleNamespace()

In [20]:
args.arch = "resnet34"
args.pretrained = True
args.sa_blur = True
args.loss = "feature_loss"
args.epochs= 1
args.save_prefix= "hello"
args.dataset="clean"
args.lr = 1e-3

In [5]:
# Getting the Architecture:
mxr34 = lambda x: mxresnet34()
mxr50 = lambda x: mxresnet50()

models_dict = dict(resnet34=models.resnet34, resnet50=models.resnet50,
                   mxresnet34=mxr34, mxresnet50=mxr50)

In [6]:
# Getting the loss function:
from feature_loss import FeatureLoss
loss_dict = dict(mse=MSELossFlat, feature_loss=FeatureLoss)

In [7]:
# Dataset
dataset_dict = dict(clean=['Train_Clean', 'Validation_Clean'],
                    realworld=['Train_RealWorld', 'Validation_RealWorld'])
stats_dict   = dict(clean=clean_stats, realworld=real_stats)

DATA = Path('../Data/')
PATH_TRAIN = dataset_dict[args.dataset][0]
PATH_VALID = dataset_dict[args.dataset][1]
norm_stats = stats_dict[args.dataset]
tfms = get_transforms(max_warp=0.0)
seed = 42

data = get_data_new(1, size="full", dataset="new", tfms=tfms, seed=seed,
                   folders=[PATH_TRAIN, PATH_VALID], stats=norm_stats)

In [21]:
model_func = models_dict[args.arch]
loss_func = loss_dict[args.loss]()
epochs = int(args.epochs)
pretrained = args.pretrained
sa_blur = args.sa_blur
lr = float(args.lr)

In [22]:
from datetime import date
today = date.today().__str__()
name  = (args.save_prefix + '-' + 
         today + '-' + 
         args.arch + '-' + 
         ("pretrained" if args.pretrained else "") + '-' + 
         ("sa_blur" if args.sa_blur else "")+ '-'  + 
         args.loss + '-' + 
         args.dataset + '-' +
         str(args.epochs))

In [23]:
print(f"Training run name : {name}")

Training run name : hello-2020-04-06-resnet34-pretrained-sa_blur-feature_loss-clean-1


In [24]:
wd = 1e-3
csv_name = "CSVs/" + name

path = "/home/ubuntu/competitions/hsi_reconstruction/Paper"
learn = unet_learner(data, model_func, pretrained=pretrained, wd=wd, loss_func=loss_func, 
                     self_attention=sa_blur, blur=True, norm_type=NormType.Weight,
                     callback_fns = [partial(SaveModelCallback, every='improvement', 
                                             monitor='MRAE', mode='min', name='best'),
                                     partial(CSVLogger, 
                                             filename=csv_name, append=True),],
                     metrics=MRAE,
                     path=path)

learn.unfreeze()
gc.collect();

In [25]:
def do_fit(save_name, lrs=slice(lr), pct_start=0.3, epochs=None):
    learn.fit_one_cycle(epochs, lrs, pct_start=pct_start, div_factor=100)
    learn.save(save_name)

In [26]:
do_fit(name, lrs=slice(lr), epochs=epochs)

epoch,train_loss,valid_loss,MRAE,time
0,0.515281,0.726001,0.284803,01:16


Better model found at epoch 0 with MRAE value: 0.2848031222820282.
